In [180]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

print (is_string_dtype(df['to']))
#print (df.head())



#print (is_numeric_dtype(df1['user']))


True


In [193]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file, curdoc

data = pd.read_csv('csv\calls.csv')

# remove missed calls
data = data[data.direction != 'Missed']
#print (data.head())
#change the direction of incoming calls
incomingRows = data[data.direction == 'Incoming']

for row in incomingRows.iterrows():
    data.set_value(row[0],'user',row[1]['other'])
    data.set_value(row[0],'other',row[1]['user'])

#remove direction
data.drop(['direction'], axis=1, inplace=True)

# rename columns
data.rename(columns={'user': 'from', 'other': 'to'}, inplace=True)

# clean the data in terms of the phone numbers
data['from'] = data['from'].astype(str)
data['to'] = data['to'].astype(str)

# 66 rows have '9' in the column 'from'. remove these rows:
data = data[data['from'] != '9']

# Some of the rows in the column 'to' have leading zeros. Remove them:
for row in data.iterrows():
        if(row[1]['to'][0] == '0'):
            data.set_value(row[0],'to',row[1]['to'][1:])
            
print (data.head())

df = data
#print (set(df['to'].str.len()))
#print (set(df['from'].str.len()))
mask= (df['to'].str.len() ==10) #| (df['to'].str.len() ==11)
df = df[mask]


         from          to  duration                       timestamp
0  7434677419  7610039694       211  Wed Sep 15 19:17:44 +0100 2010
1  7641036117  1666472054        31  Mon Feb 11 07:18:23 +0000 1980
2  7371326239  7641036117        45  Mon Feb 11 07:45:42 +0000 1980
3  7641036117  7681546436        10  Mon Feb 11 08:04:42 +0000 1980
4  7641036117  7681546436         0  Mon Feb 11 08:05:31 +0000 1980


In [148]:
df2 = df1
print (df1.head())
df2['timestamp']=pd.to_datetime(df2['timestamp'],format='%d%b%Y %H:%M',errors='coerce')
print (df2.head())

         user       other  duration                       timestamp
0  7610039694  7434677419       211  Wed Sep 15 19:17:44 +0100 2010
1  7641036117  7371326239        45  Mon Feb 11 07:45:42 +0000 1980
2  7641036117  7681546436         0  Mon Feb 11 08:06:18 +0000 1980
3  7641036117  7588304495       124  Thu Sep 09 19:35:37 +0100 2010
4  7163185791  7230262224        47  Tue Sep 14 15:28:16 +0100 2010
         user       other  duration timestamp
0  7610039694  7434677419       211       NaT
1  7641036117  7371326239        45       NaT
2  7641036117  7681546436         0       NaT
3  7641036117  7588304495       124       NaT
4  7163185791  7230262224        47       NaT


In [192]:
from sklearn.cluster import KMeans,DBSCAN,Birch
from bokeh.models import ColumnDataSource
output_notebook()
def KMeansclustering(df2,features_selected,clusters):
    model=KMeans(n_clusters=clusters,random_state=0)
    model_fit = model.fit_predict(df2[features_selected])
    return model


def getColors(model):
    colors=[]
    for i in model.labels_:
        if i==0:
            colors.append('red')
        if i==1:
            colors.append('blue')
        if i==2:
            colors.append('green')
        if i==3:
            colors.append('purple')
        if i==4:
            colors.append('brown')
        if i==5:
            colors.append('yellow')
    return colors

featuresList=['from','duration']
#KMeans clustering
model = KMeansclustering(df,featuresList,5)
model2 = DBSCAN(eps=20,min_samples=2)
modelFit2 = model2.fit_predict(df[featuresList])
colorKMeans = getColors(model)
#plot
p=figure(plot_height=200, plot_width=500, title='KMeans')
actualData1 = ColumnDataSource(data=dict(x=df.loc[:,'to'],y=df.loc[:,'from'], colors=colorKMeans))
p.circle('x','y',fill_color='colors',line_color='colors',source=actualData1)
p.xaxis.axis_label = 'to'
p.yaxis.axis_label = 'from'
show(p)


Loading BokehJS ...